<h1>Getting started with TensorFlow</h1>
<p>The following notebook assumes limited to no knowledge of TensorFlow, but it will require basic Python literacy and for the TensorFlow and Numpy modules to be installed (They should be unless you maliciously uninstalled them.</p>

In [1]:
import tensorflow as tf
import numpy as np

/home/student/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


<a name="Reset"></a>
<h2>Resetting the TensorFlow graph.</h2>
<p>The following code resets the TensorFlow graph. This is typically run before creating your networks to ensure that there are no duplicate tensors.</p>

In [2]:
tf.reset_default_graph()